In [8]:
import glob
import pandas as pd
import fnmatch
import os
import re
from dateutil import parser
from dateutil.parser import ParserError

# Patterns to match Excel files in the data directory and its subdirectories
data_patterns = ['data/*.xlsx', 'data/**/*.xlsx']

# Initialize an empty DataFrame
combined_df = pd.DataFrame()


def extract_date_from_filename(filename):
    # Define possible date patterns
    date_patterns = [
        r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
        r'\d{2}-\d{2}-\d{4}',  # DD-MM-YYYY or MM-DD-YYYY
        r'\d{2}_\d{2}_\d{4}',  # DD_MM_YYYY or MM_DD_YYYY
        r'\d{1,2}-\d{1,2}-\d{2,4}',  # D-M-YY or MM-DD-YY
        r'\d{1,2}_\d{1,2}_\d{2,4}',  # D_M_YY or MM_DD_YY
        r'\d{8}',  # YYYYMMDD
        r'\d+\.\d+\.\d+'  # Dates with periods (e.g., 01.02.2023)
    ]
    for pattern in date_patterns:
        matches = re.findall(pattern, filename)
        for date_str in matches:
            try:
                parsed_date = parser.parse(date_str, dayfirst=False)
                return parsed_date.strftime('%Y-%m-%d')
            except ParserError:
                continue
    print(f"No valid date found in filename: {filename}")
    return 'Unknown'


def split_filename(filename):
    parts = filename.split("_")
    return parts[:3]  # Only return the first three parts


# Collect all Excel files from the data directory and its subdirectories
excel_files = []
for pattern in data_patterns:
    excel_files.extend(glob.glob(pattern, recursive=True))

# Remove duplicates in case of overlap
excel_files = list(set(excel_files))

# Using glob to find all xlsx files in the main directory and all subdirectories
for filepath in excel_files:
    print(f"Checking file: {filepath}")
    try:
        xls = pd.ExcelFile(filepath)
        print(f"Sheets available in {filepath}: {xls.sheet_names}")
        sheet_name = next(
            (sheet for sheet in xls.sheet_names if fnmatch.fnmatch(sheet, '*')),
            None
        )
        if sheet_name:
            df = pd.read_excel(filepath, sheet_name=sheet_name)
            # Standardize column names
            df.columns = df.columns.str.strip().str.lower()
            # Print column names for debugging
            print(f"Columns in sheet {sheet_name}: {df.columns.tolist()}")
            if 'metoverall' in df.columns or 'metaccess' in df.columns:
                # Combine 'metoverall' and 'metaccess' into a single column
                if 'metoverall' in df.columns:
                    df['metcombined'] = df['metoverall']
                else:
                    df['metcombined'] = df['metaccess']
                df['metcombined'] = df['metcombined'].map({1: 1, 0: 0})     # map "Y" to 1 and "N" to 0 -- df['metcombined'] = df['metcombined'].map({"Y": 1, "N": 0})
                # Define column sets
                required_columns = ['state', 'county', 'metcombined']                   # required_columns = ['countyname', 'metcombined']
                opt_columns = {
                    'provider_or_facility': ['facility or provider', 'provider or facility'],   # determine columns to search for
                    'provider_specialty': ['providertype', 'specdesc']                          # determine columns to search for
                }   

                # Check and add optional columns
                for key, cols in opt_columns.items():
                    col_to_add = next((col for col in cols if col in df.columns), None)
                    if col_to_add:
                        df[key] = df[col_to_add]

                # Ensure required columns are present
                subset_df = df[required_columns].copy()

                # Add optional columns
                for key in opt_columns.keys():
                    if key in df.columns:
                        subset_df[key] = df[key]

                # Extract date and parts from the filename
                subset_df['date'] = extract_date_from_filename(filepath)
                parts = split_filename(os.path.basename(filepath))
                for i in range(len(parts)):
                    subset_df[f'part_{i + 1}'] = parts[i] if len(parts) > i else 'Unknown'

                combined_df = pd.concat([combined_df, subset_df], ignore_index=True)
            else:
                print(f"'MetOverall' or 'MetAccess' column not found in sheet {sheet_name}.")
        else:
            print(f"No matching sheet name found in {filepath}.")
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
print("Combined DataFrame:")
print(combined_df)
# Save the merged DataFrame to a CSV file
combined_df.to_csv('combined_adequacy_detail.csv', index=False)

Checking file: data\TX_CAID_Network3 90%_07.01.24.xlsx
Sheets available in data\TX_CAID_Network3 90%_07.01.24.xlsx: ['CAID']
Columns in sheet CAID: ['state', 'county', 'facility or provider', 'providertype', 'metaccess']
Checking file: data\TX_CAID_Network3 90%_08.15.24.xlsx
Sheets available in data\TX_CAID_Network3 90%_08.15.24.xlsx: ['CAID']
Columns in sheet CAID: ['state', 'county', 'facility or provider', 'providertype', 'metaccess']
Checking file: data\TX_MA_H09156 Network2_08.2.24.xlsx
Sheets available in data\TX_MA_H09156 Network2_08.2.24.xlsx: ['MA']
Columns in sheet MA: ['state', 'county', 'provider or facility', 'specdesc', 'metaccess', 'metproviders', 'metoverall']
Checking file: data\TX_MA_H09156 Network2_07.15.24.xlsx
Sheets available in data\TX_MA_H09156 Network2_07.15.24.xlsx: ['MA']
Columns in sheet MA: ['state', 'county', 'provider or facility', 'specdesc', 'metaccess', 'metproviders', 'metoverall']
Combined DataFrame:
     state       county  metcombined provider_or_f

In [6]:
import qgrid
qgrid.show_grid(combined_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [9]:
# Remove or comment out the qgrid-related lines
# import qgrid
# qgrid.show_grid(combined_df)

# Instead, use these lines to display your DataFrame
print(combined_df.head())  # Display the first 5 rows
print("\nDataFrame Info:")
print(combined_df.info())  # Display information about the DataFrame

# If you want to see all columns, you can use:
# pd.set_option('display.max_columns', None)
# print(combined_df)

# If you want to save the DataFrame to a CSV file for easier viewing:
combined_df.to_csv('combined_df_output.csv', index=False)
print("\nDataFrame has been saved to 'combined_df_output.csv'")

  state       county  metcombined provider_or_facility  \
0    TX     Amarillo            1             Provider   
1    TX      El Paso            1             Provider   
2    MT     Missoula            1             Provider   
3    TX      Lubbock            1             Facility   
4    NM  Albuquerque            0             Provider   

       provider_specialty        date part_1 part_2        part_3  
0      Forensic Pathology  2024-07-01     TX   CAID  Network3 90%  
1  Allergy and Immunology  2024-07-01     TX   CAID  Network3 90%  
2       Internal Medicine  2024-07-01     TX   CAID  Network3 90%  
3              Cardiology  2024-07-01     TX   CAID  Network3 90%  
4  Allergy and Immunology  2024-07-01     TX   CAID  Network3 90%  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   state          